In [5]:
import json

# Load the Yosys-generated JSON netlist
with open('synthesized_4x4.json', 'r') as f:
    netlist = json.load(f)

# Specify the top module manually (adjust the name as needed)
top_module_name = 'Multiplier_4x4'
if top_module_name not in netlist.get('modules', {}):
    raise KeyError(f"Module {top_module_name} not found in the netlist.")

module = netlist['modules'][top_module_name]

# Prepare a circuit dictionary with a set for nodes and a list for gates.
circuit = {"nodes": set(), "gates": []}

# Define a mapping from Yosys cell names to the gate types you want.
# You can add more mappings or adjust as needed.
cell_mapping = {
    "$_AND_": "AND",
    "$_OR_": "OR",
    "$_NOT_": "NOT",
    "$_ANDNOT_": "ANDNOT",
    "$_ORNOT_": "ORNOT",
    "$_NAND_": "NAND",
    "$_NOR_": "NOR",
    "$_XOR_": "XOR",
    "$_XNOR_": "XNOR",
    # Add additional mappings if desired
}

# Iterate over each cell in the top module.
for cell_name, cell in module.get("cells", {}).items():
    # Get the cell type and map it if necessary.
    yosys_type = cell["type"]
    gate_type = cell_mapping.get(yosys_type, yosys_type)

    # Build a list of nodes (nets) for this cell.
    # The "connections" field holds the port-to-net mapping.
    gate_nodes = []
    for port, nets in cell.get("connections", {}).items():
        # In Yosys JSON, nets is usually a list (even for single connections).
        if isinstance(nets, list):
            # Here we assume each port is connected to one net,
            # but adjust if ports have multiple nets.
            net = nets[0]
        else:
            net = nets
        gate_nodes.append(net)
        # Add the net to our circuit's node set.
        circuit["nodes"].add(net)
    
    # Append this gate to the circuit list.
    circuit["gates"].append({"type": gate_type, "nodes": gate_nodes})

# Convert nodes from a set to a sorted list (for reproducibility)
circuit["nodes"] = sorted(list(circuit["nodes"]))

# Now 'circuit' is in the format your generate_weight_from_logic.py expects.
print(json.dumps(circuit, indent=4))


{
    "nodes": [
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
        32,
        33,
        34,
        35,
        36,
        37,
        38,
        39,
        40,
        41,
        42,
        43,
        44,
        45,
        46,
        47,
        48,
        49,
        50,
        51,
        52,
        53,
        54,
        55,
        56,
        57,
        58,
        59,
        60,
        61,
        62,
        63,
        64,
        65,
        66,
        67,
        68,
        69,
        70,
        71,
        72,
        73,
        74,
        75,
        76,
        77,
        78,
        79,
        80,
        81,
        82,
        83,
       